In [1]:
import nltk as tk
tk.download('punkt')

import copy
import random
import collections
import os

from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jacop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def load_line_to_ids_dict(fname):
        """Loads a vocabulary file into a dictionary."""
        vocab = collections.OrderedDict()
        with open(fname, "r", encoding="utf-8") as reader:
            chars = reader.readlines()
        for index, char in enumerate(chars):
            char = char.rstrip('\n')
            vocab[char] = index
        return vocab

In [3]:
def create_adv_word(orig_token, rng, debug = False):
    # HERE IS WHERE THE MAGIC HAPPENS
    
    char2ids_dict = load_line_to_ids_dict("./data/dict/bert_char_vocab")
    
    token = list(copy.deepcopy(orig_token))
    #se la parola è più corta di 4 caratteri, aggiungo un carattere a caso
    if len(orig_token) < 4:
        rand_idx = rng.randint(0, 80) #le prime 80 sono caratteri sensati
        rand_char = list(char2ids_dict.keys())[rand_idx] #prendo un carattere a caso
        insert_idx = rng.randint(0, len(orig_token)-1) #prendo un indice a caso tra 0 e la lunghezza della parola
        token = token[:insert_idx] + [rand_char] + token[insert_idx:] #inserisco il carattere tra i due indici
        if debug:
            print(f"Insert the char:{rand_char} orig_token: {orig_token} new_token: {token}")
    
    #se la parola è più lunga di 4 caratteri, scelgo se cancellare o scambiare un carattere
    else:
        if rng.random() < 0.5: #con prob del 50% tolgo un carattere a caso
            rand_idx = rng.randint(0, len(orig_token)-1)
            del token[rand_idx]
            if debug:
                print(f"Delete the char:{orig_token[rand_idx]} orig_token: {orig_token} new_token: {token}")
        else: #con prob del 50% scambio due caratteri a caso
            idx = random.randint(1, len(orig_token)-2)
            token[idx], token[idx+1] = token[idx+1], token[idx]
            if debug:
                print(f"Swap the char:{token[idx:idx+2]} orig_token: {orig_token} new_token: {token}")
    token = ''.join(token)
    return token

In [6]:
rnd = random.Random(42)
for w in ['can', 'ram', 'len', 'sia']:
    print(create_adv_word(w, rng = rnd))

mcan
Bram
Slen
siua


In [9]:
def create_adv_ds(sentece, rng, debug = False):
    
    num_diff = num_same = 0
    adv_probability = 0.10
    
    doc_tokens = tk.word_tokenize(sentece)
    processed_tokens = []
    for idx, token in enumerate(doc_tokens):
        ori_token = copy.deepcopy(token)
        if rng.random() < adv_probability: #con probabilità "adv_proba" cambio la parola
            token = create_adv_word(token, rng, debug = debug)
            
        if ori_token != token:
            #if num_diff % 1000 == 0:
            #    print(f"Change the token {ori_token} To {token}")
            num_diff += 1
        else:
            num_same += 1
        
        processed_tokens.append(token)
            
    print(f"num_same: {num_same}, num_diff: {num_diff}")
    #print(f"tokenized doc: {' '.join(tokenized_tokens)}")
        
    return TreebankWordDetokenizer().detokenize(processed_tokens), num_same, num_diff

create_adv_ds("I am a student.",rng = rnd, debug=True)[0]

Delete the char:t orig_token: student new_token: ['s', 'u', 'd', 'e', 'n', 't']
num_same: 4, num_diff: 1


'I am a sudent.'

In [10]:
import pandas as pd
test_df = pd.read_csv("./data/glue_dataset/SST2/test.tsv", sep = "\t")
test_df.head()

,sentence,label,idx
0,uneasy mishmash of styles and genres .,-1,0
1,this film 's relationship to actual tension is...,-1,1
2,"by the end of no such thing the audience , lik...",-1,2
3,director rob marshall went out gunning to make...,-1,3
4,lathan and diggs have considerable personal ch...,-1,4


In [11]:
len(test_df)

1821

In [12]:
rng = random.Random(42) #seed = 42

adv_test_df = test_df.copy(deep=True)

#adv_test_df['sentence'] = adv_test_df['sentence'].apply(lambda x: create_adv_ds(x, debug = True)[0])

num_diff = num_same = 0
num_same = num_diff = 0
for idx, row in adv_test_df.iterrows():
    adv_test_df.at[idx,'sentence'], tmp_num_sam, tmp_num_diff = create_adv_ds(row['sentence'], rng)
    #row['sentence'] = sent
    num_same += tmp_num_sam
    num_diff += tmp_num_diff

num_same: 6, num_diff: 1
num_same: 25, num_diff: 6
num_same: 18, num_diff: 3
num_same: 12, num_diff: 0
num_same: 18, num_diff: 1
num_same: 12, num_diff: 0
num_same: 22, num_diff: 1
num_same: 22, num_diff: 4
num_same: 15, num_diff: 3
num_same: 7, num_diff: 2
num_same: 12, num_diff: 1
num_same: 21, num_diff: 1
num_same: 18, num_diff: 4
num_same: 14, num_diff: 3
num_same: 16, num_diff: 0
num_same: 21, num_diff: 1
num_same: 20, num_diff: 2
num_same: 8, num_diff: 2
num_same: 21, num_diff: 0
num_same: 6, num_diff: 0
num_same: 22, num_diff: 2
num_same: 11, num_diff: 0
num_same: 22, num_diff: 1
num_same: 3, num_diff: 0
num_same: 24, num_diff: 3
num_same: 18, num_diff: 2
num_same: 15, num_diff: 3
num_same: 38, num_diff: 2
num_same: 23, num_diff: 1
num_same: 17, num_diff: 1
num_same: 8, num_diff: 0
num_same: 18, num_diff: 5
num_same: 6, num_diff: 1
num_same: 10, num_diff: 1
num_same: 22, num_diff: 2
num_same: 38, num_diff: 2
num_same: 31, num_diff: 2
num_same: 7, num_diff: 0
num_same: 21, num_di

In [13]:
tot = num_diff + num_same
print(f"same: {num_same/tot:.2f}, diff: {num_diff/tot:.2f}")

same: 0.90, diff: 0.10


In [14]:
adv_test_df.head(3)

,sentence,label,idx
0,uneasy mismash of styles and genres.,-1,0
1,tis film 'Ws relationship to actual tesnion is...,-1,1
2,"by the end of no suhc thing the audience, like...",-1,2


In [15]:
test_df.head(3)

,sentence,label,idx
0,uneasy mishmash of styles and genres .,-1,0
1,this film 's relationship to actual tension is...,-1,1
2,"by the end of no such thing the audience , lik...",-1,2


In [ ]:
#uncomment to save on disk
#adv_test_df.to_csv(".\outdir", sep = '\t', index = False)